## Project 2 
#### Name : Dnyanai Surkutwar
#### Email: dsurkutwar@scu.edu


df2 = https://data.wprdc.org/dataset/7fb0505e-8e2c-4825-b22c-4fbee8fc8010/resource/2d963e35-4f69-495e-985e-55acd72c87ca/download/crimelabaccidentaldrugdeathsextract2017.csv: refers to the accidental deaths in CT state.

df1 = https://data.ct.gov/api/views/rybz-nyjw/rows.csv?accessType=DOWNLOAD : refers to the the Allegheny county Drug related deaths. 

## Importing required libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from math import pi
from functools import reduce
import matplotlib.image as mpimg


pd.set_option('display.max_columns',50)

In [ ]:
df1 = pd.read_csv('./Accidental_Drug_Related_Deaths_2012-2018.csv')
df1.head()

## Data Preparation:
### 1) Replacing the N/Y values with 0/1
### 2) Making the datatypes uniform in the dataframe

In [ ]:
#We are changing the Y values in the cols to 1 and NaN to 0.
df1.replace('Y',1,inplace=True) # Replaces Y to 1 and inplace makes the change permanent to the df
df1.fillna(0,inplace=True) 

In [ ]:
df1['Date'] = pd.to_datetime(df1['Date'])

In [ ]:
df1['Time'] = df1['Date'].dt.time
df1.drop(['Time'],axis=1,inplace=True)

In [ ]:
df1['Year']= df1['Date'].dt.year.astype('Int64')
df1.head(1)

In [ ]:
#Need to change the columns of float to int as its unecc.
df1.Age = df1.Age.astype('int64')
df1.Heroin= df1.Heroin.astype('int64')
df1.Cocaine = df1.Cocaine.astype('int64')
df1.FentanylAnalogue = df1.FentanylAnalogue.astype('int64')
df1.Oxycodone = df1.Oxycodone.astype('int64')
df1.Oxymorphone = df1.Oxymorphone.astype('int64')
df1.Ethanol = df1.Ethanol.astype('int64')
df1.Hydrocodone = df1.Hydrocodone.astype('int64')
df1.Benzodiazepine = df1.Benzodiazepine.astype('int64')
df1.Methadone = df1.Methadone.astype('int64')
df1.Amphet = df1.Amphet.astype('int64')
df1.Tramad = df1.Tramad.astype('int64')
df1.Hydromorphone = df1.Hydromorphone.astype('int64')
df1.OpiateNOS = df1.OpiateNOS.astype('int64')
df1.Race = df1.Race.astype('str')
df1.head()

In [ ]:
#There are some unknown and 0 values in this dataframe because of which
#the graphs are not properly plotted.,lets remove those.

df1.drop(df1[(df1['Sex'].values=='Unknown')].index,axis=0,inplace=True)
df1.drop(df1[(df1['Sex'].values==0)].index,axis=0,inplace=True)

In [ ]:
#Our data is specificaly from 2012 to 2018 so remove the datapoint 
#which is not in this range and maybe wrong/redundant.

df1.drop(df1[(df1['Year'].values==1970)].index,axis=0,inplace=True)

In [ ]:
#Converting everything to lower as we have many str values thus to 
#avoid errors we make everything in lowercase. 
df1['DeathCity'] = df1['DeathCity'].str.lower()
df1['ResidenceCity'] = df1['ResidenceCity'].str.lower()
df1['ResidenceCounty'] = df1['ResidenceCounty'].str.lower()
df1['DeathCounty'] = df1['DeathCounty'].str.lower()
df1['Location'] = df1['Location'].str.lower()
df1['DescriptionofInjury'] = df1['DescriptionofInjury'].str.lower()

In [ ]:
#There are still few cols which are not int64, so need to convert them.

##Since bool objects cannot be replaced, need to convert to str then int
## Y-A, Y POPS are all in teh dataset, converting it to 1
df1.Fentanyl = df1.Fentanyl.astype('str')
df1['Fentanyl'].replace(['Y-A','Y POPS','Y (PTCH)'],1,inplace=True)
df1.Fentanyl = df1.Fentanyl.astype('int64')

##Chaning and replacing for Morphine_Heroin
df1.Morphine_NotHeroin = df1.Morphine_NotHeroin.astype('str')
df1.Morphine_NotHeroin.replace(['YES','STOLE MEDS','NO RX BUT STRAWS','PCP NEG'],1,inplace=True)
df1.Morphine_NotHeroin.replace('N',0,inplace=True)
df1.Morphine_NotHeroin = df1.Morphine_NotHeroin.astype('int64')

##For AnyOpioid
df1.AnyOpioid = df1.AnyOpioid.astype('str')
df1.AnyOpioid.replace('N',0,inplace=True)
df1.AnyOpioid = df1.AnyOpioid.astype('int64')

df1.dtypes

##As per our steps earlier, we have to again remove 1970 year from the 
## column year as we are primarily foucusing on 2012-2018
df1.drop(df1[(df1['Year'].values==1970)].index,axis=0,inplace=True)
         

In [ ]:
##Here, we are creating a concatmask to identify which drug was involved.
##Thus, we just convert every column to str and concat it to the next drug col value.

df1['ConcatResult'] = df1.Heroin.astype(str)+df1.Cocaine.astype(str)+df1.Fentanyl.astype(str)+\
df1.FentanylAnalogue.astype(str)+df1.Oxycodone.astype(str)+df1.Oxymorphone.astype(str)+\
df1.Ethanol.astype(str)+df1.Hydrocodone.astype(str)+df1.Benzodiazepine.astype(str)+\
df1.Methadone.astype(str)+df1.Amphet.astype(str)+df1.Tramad.astype(str)+df1.Morphine_NotHeroin.astype(str)+\
df1.Hydromorphone.astype(str)+df1.OpiateNOS.astype(str)+df1.AnyOpioid.astype(str)

df1['NumberOfDrugs'] = df1.Heroin+df1.Cocaine+df1.Fentanyl+df1.FentanylAnalogue+df1.Oxycodone+\
df1.Oxymorphone+df1.Ethanol+df1.Hydrocodone+df1.Benzodiazepine+df1.Methadone+\
df1.Amphet+df1.Tramad+df1.Morphine_NotHeroin+df1.Hydromorphone+df1.OpiateNOS+df1.AnyOpioid

##Here we created and added two new columns to df2.

In [ ]:
df1.head()

## 1) Doing classification using decision tree. 

## ## Interesting Fact 1: 
### Classification model is a supervised model which is used to predict the next possible data point based on what is your target variable and what independent variables it depends on.
### The accuracy of the model below is 32% which is not too good, this can be adjusted by tuning the parameters and optimizing the tree.

In [ ]:
#Making a new dataset and taking only those columns which we need to look at
df2 = df1[['Sex','Age','Heroin','Cocaine','NumberOfDrugs']].copy()

In [ ]:
#Classification library import
import sklearn as sk
import sklearn.tree as tree
from IPython.display import Image  
import pydotplus 
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

## Making dummies for the other columns besides the target column

In [ ]:
df = pd.get_dummies(df2, columns=['Sex','Age','Heroin','Cocaine'],dummy_na=False) ##getdummies creates 1/0 dummy values 
##for all categorial/numerical values 

In [ ]:
df.head()

## We will divide our dataset into training and testing data and first train it and then test it on a new data

In [ ]:
X = df.drop('NumberOfDrugs',axis=1)

In [ ]:
Y = df.NumberOfDrugs

In [ ]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

In [ ]:
## Setting the classifier
dt = tree.DecisionTreeClassifier(max_depth=2) 

In [ ]:
dt.fit(X_train,y_train)

In [ ]:
#Predict the response for test dataset
y_pred = dt.predict(X_test)

In [ ]:
# Model Accuracy, how often is the classifier correct?
roundNo = round((metrics.accuracy_score(y_test, y_pred)*100),2)
print("Accuracy:",roundNo)

## Plotting the Decision Tree

In [ ]:
# This code will visualize a decision tree dt, trained with the attributes in X and the class labels in Y
dt_feature_names = list(X.columns) #Converting the independent variables to a list 
dt_target_names = [str(s) for s in Y.unique()] #take unique values of Y i.e. DogName
tree.export_graphviz(dt, out_file='tree.dot', 
    feature_names=dt_feature_names, class_names=dt_target_names,
    filled=True)  
graph = pydotplus.graph_from_dot_file('tree.dot') #generating a tree
graph.set_size('"20,25!"')
Image(graph.create_png())


## ## Interesting Fact 2: 
###  The hartford city & hartford county has the highest number of death contributing almost 31% of the entire states drug related deaths.Also, gender wise precentage for hartford city alone is 2.34% Male involving 2 and 3 drugs. The city death has highest number of whites. 
### The hartford city has an all time high count since 2016, increasing each year by 11-12% since 2016. 



In [ ]:
 df3 = df1[['Year','Age','Sex','Race','DeathCity','ResidenceCity','NumberOfDrugs']].copy()

## For the above columns we are checking for na and dropping them if they are insignificant

In [ ]:
df3.Age.isna().sum()

In [ ]:
df3.Sex.isna().sum()

In [ ]:
df3.Race.isna().sum()

In [ ]:
print('RseidenceCity Total: ',df3.ResidenceCity.size)
print('ResidenceCity null/NaN values: ',df3.ResidenceCity.isna().sum())
### Thus, we are deleting them 
df3.ResidenceCity.dropna(inplace=True)

In [ ]:
print('RseidenceCity Total: ',df3.DeathCity.size)
print('ResidenceCity null/NaN values: ',df3.DeathCity.isna().sum())
### Thus, we are deleting them 
df3.DeathCity.dropna(inplace=True)
df3.drop(df1[(df1['DeathCity'].values=='06340')].index,axis=0,inplace=True)

In [ ]:
# Here we are finding out which DeathCity is the highest in which year
df3.groupby('DeathCity')['Year'].value_counts().nlargest(15)

In [ ]:
#Plotting the above findings
df3.groupby('DeathCity')['Year'].value_counts().nlargest(10).unstack().plot(kind='bar')

In [ ]:
## Here, we are finding out the max @ hartford for NumberofDrugs involved according to their sex, we are taking out the %
total = df3.DeathCity.count()
round(((df3[df3.DeathCity=='hartford'].groupby('NumberOfDrugs')['Sex'].value_counts().nlargest(5)/total)*100),2)

In [ ]:
# Here, we are finding out the dominant race in the deaths recorded.
df3[df3.DeathCity=='hartford'].Race.value_counts().nlargest(5).plot(kind='bar')
print('The race with the highest death count is: ')

In [ ]:
round(df1.DeathCounty.value_counts()/df1.DeathCounty.count()*100,2)

## ## Interesting fact 3

### Lets do a clustering model and find out which is the race with the higest involvement with the highest drug related deaths.

### Clustering is an unsupervised algorithm which groups similar data points together an aims to increase inter cluster distance and decrease intra cluster distance. 

### As per our observations below, the White race is predominantly present in all cluster labels, this tells us that almost all kinds of age and Sex data points have similarity with the "White" race. Moreover, we also see how the clusters are divided based on the deceased ages, and cluster number 4 & 2 refering to ages 54-74yrs & 14-32yrs are the most dominant ones. 

### Creating dummy variables for ML model

In [ ]:
#Dropping a vlue which is invalid for Race
#df3.drop(df1[(df1['DeathCity'].values=='06340')].index,axis=0,inplace=True)
df3.drop(df3[(df3['Race'].values =='0')].index,axis=0,inplace=True)

In [ ]:
#Create dummy variables for the DogName column values.
data = pd.get_dummies(df3)
data.head()

### Here, we are importing the scikit-learn modules to use in K-Mean Clustering model

In [ ]:
from sklearn import metrics
from sklearn.cluster import KMeans

bestSil = -1
for k in range(2,6):
    print ('k= %d ' % k)
    #We are using the KMeans clustering model
    clus = [KMeans(n_clusters=k)]
    for cl in clus:
        res = cl.fit(data) #Here, we are fitting the model here to get accurate values
        sil = metrics.silhouette_score(data, res.labels_)#Generate the silhouette coefficient 
        print (str(cl)[:6] + ' with k=' +str(k) + ": " + str(round(sil,4)))
        if (sil > bestSil):
            bestSil = sil
            bestCl = cl
            bestK = k
print('***********************************************')
print ('Best output is with k=' +str(bestK) )
print('**********************')
print ('With Silhouette Score ' + str(round(bestSil,4)))

In [ ]:
df3['Cluster_labels'] = cl.labels_+1 ##Adding 1 to begin it from 1

In [ ]:
df3.groupby('Cluster_labels')['Race'].value_counts().nlargest(15)

In [ ]:
sns.relplot(x="Cluster_labels", y="Age", data=df3)

## In the above graph, we see that the cluster labels have clear age limits defined. 
### Cluster 1 is refering to ages 48-58yrs,
### Cluster 2 is refering to ages 15-32yrs,
### Cluster 3 is refering to ages 32-40yrs,
### Cluster 4 is refering to ages 56-76yrs,
### Cluster 5 is refering to ages 39-48yrs.
### As we can observe, the maximum death age lies in Cluster 2 & 4. 

In [ ]:
sns.factorplot(y='Race', data=df3, x='Age',\
               aspect=2, kind='point')

## Importing a new dataframe into ds about allengheny county for year 2017. 

In [ ]:
ds = pd.read_csv('./crimelabaccidentaldrugdeathsextract2017.csv')
ds.head()

In [ ]:
## pd.to_datetime converts to datetime
ds['Death Date'] = pd.to_datetime(ds['Death Date'])

In [ ]:
ds['Heroin'] = ds['Combined OD1'][ds['Combined OD1'] == 'Heroin'].apply(lambda x:1 if (x=='Heroin') else (0))
ds['Cocaine'] = ds['Combined OD1'][ds['Combined OD1'] == 'Cocaine'].apply(lambda x:1 if (x=='Cocaine') else (0))
ds.fillna(0,inplace=True)
ds.head()

## Converting to 1 for every column having Heroin or Cocaine in it

In [ ]:
ds['Heroin'].loc[ds['Combined OD2']=='Heroin'] = 1
ds['Heroin'].loc[ds['Combined OD3']=='Heroin'] = 1
ds['Heroin'].loc[ds['Combined OD4']=='Heroin'] = 1
ds['Heroin'].loc[ds['Combined OD5']=='Heroin'] = 1
ds['Heroin'].loc[ds['Combined OD6']=='Heroin'] = 1
ds['Heroin'].loc[ds['Combined OD7']=='Heroin'] = 1

In [ ]:
ds['Cocaine'].loc[ds['Combined OD2']=='Cocaine'] = 1
ds['Cocaine'].loc[ds['Combined OD3']=='Cocaine'] = 1
ds['Cocaine'].loc[ds['Combined OD4']=='Cocaine'] = 1
ds['Cocaine'].loc[ds['Combined OD5']=='Cocaine'] = 1
ds['Cocaine'].loc[ds['Combined OD6']=='Cocaine'] = 1
ds['Cocaine'].loc[ds['Combined OD7']=='Cocaine'] = 1

In [ ]:
ds.Heroin = ds.Heroin.astype('Int64')
ds.Cocaine = ds.Cocaine.astype('Int64')

In [ ]:
## Making a new year column for df2 to access only the year
ds['Year']= ds['Death Date'].dt.year.astype('Int64')

## After data preparation we will merge the datasets 

In [ ]:
ds1 = ds[['Year','Age','Sex','Race','Heroin','Cocaine']].copy()
print(ds1.shape)
ds1.head()

## These two datasets are not ideal for merging as Year is the only column in common. 


In [ ]:
dfmerge = df5.merge(ds1,left_on='Year',right_on='Year',how='inner').drop_duplicates(inplace=True)